In [1]:
import sqlite3
import os
from sqldb import SqlDatabase

In [2]:
sql_agent=SqlDatabase(db_name="telecom_churn", db_path="sqldatabase")
sql_agent.execute_query(query="SELECT * from customer_data limit 10;")

Database telecom_churn connected at sqldatabase


,childreninhh,handsetrefurbished,handsetwebcapable,truckowner,rvowner,homeownership,buysviamailorder,respondstomailoffers,optoutmailings,nonustravel,...,revenue_per_minute,total_calls,avg_call_duration,service_tenure,customer_support_interaction,handsetprice,incomegroup,customerid,churn,prediction
0,no,no,no,yes,yes,known,no,yes,yes,no,...,0.109302,128.4,2.426584,25.0,0.0,0.0,5.0,3114822.0,0.0,0.376834
1,no,no,yes,yes,no,known,no,no,no,no,...,0.194805,0.3,117.692308,33.0,0.0,0.0,4.0,3039574.0,0.0,0.252063
2,yes,no,yes,no,no,known,yes,yes,no,no,...,0.185745,35.0,9.111111,27.0,1.7,30.0,5.0,3059830.0,1.0,0.503596
3,yes,no,yes,yes,yes,known,yes,yes,no,no,...,0.040769,2.0,116.666667,16.0,0.0,0.0,7.0,3243582.0,1.0,0.187681
4,yes,no,yes,yes,yes,known,yes,yes,no,no,...,0.172857,13.6,11.917808,26.0,0.0,30.0,8.0,3087070.0,0.0,0.231407
5,no,no,yes,no,no,unknown,no,no,no,no,...,0.070692,274.7,9.372506,14.0,11.7,130.0,5.0,3256546.0,0.0,0.098959
6,no,no,yes,no,no,unknown,no,no,no,no,...,0.169192,11.6,20.555556,12.0,0.0,0.0,0.0,3250586.0,1.0,0.217000
7,no,no,yes,no,no,unknown,no,no,no,no,...,0.053511,113.6,9.956370,8.0,19.0,0.0,0.0,3376190.0,0.0,0.189424
8,yes,yes,yes,no,no,known,no,no,no,no,...,0.074359,105.4,5.704887,10.0,23.7,150.0,5.0,3347818.0,0.0,0.172266
9,no,yes,no,no,no,known,no,no,no,no,...,0.121814,37.4,10.885417,13.0,0.0,0.0,4.0,3236530.0,0.0,0.240121


In [3]:

# sql_agent.execute_query(query="SELECT * from customer_data limit 10;")
print(sql_agent.get_all_schemas())

Tables:

customer_data
customer_shap_contributions
customer_counterfactuals
data_dictionary
global_shap_summary


Schemas:


customer_data:
  Name: childreninhh, Type: TEXT
  Name: handsetrefurbished, Type: TEXT
  Name: handsetwebcapable, Type: TEXT
  Name: truckowner, Type: TEXT
  Name: rvowner, Type: TEXT
  Name: homeownership, Type: TEXT
  Name: buysviamailorder, Type: TEXT
  Name: respondstomailoffers, Type: TEXT
  Name: optoutmailings, Type: TEXT
  Name: nonustravel, Type: TEXT
  Name: ownscomputer, Type: TEXT
  Name: hascreditcard, Type: TEXT
  Name: newcellphoneuser, Type: TEXT
  Name: notnewcellphoneuser, Type: TEXT
  Name: ownsmotorcycle, Type: TEXT
  Name: madecalltoretentionteam, Type: TEXT
  Name: creditrating, Type: TEXT
  Name: prizmcode, Type: TEXT
  Name: occupation, Type: TEXT
  Name: maritalstatus, Type: TEXT
  Name: service_city, Type: TEXT
  Name: monthlyrevenue, Type: REAL
  Name: monthlyminutes, Type: REAL
  Name: totalrecurringcharge, Type: REAL
  Name: directora

In [62]:
import google.generativeai as genai
import google.ai.generativelanguage as glm
import pandas as pd
import numpy as np
import tabulate

In [5]:
gmodel=genai.GenerativeModel(model_name="gemini-1.5-flash-001")

In [6]:
import re

def remove_sql_and_backticks(input_text):
    """
    Removes 'sql' and '```' from the input string.

    Parameters
    ----------
        input_text : str
            the input string

    Returns
    -------
        str
            the modified string
    """
    modified_text = re.sub(r'```|sql', '', input_text)
    modified_text = re.sub(r'\\\\', '', input_text)
    return modified_text

In [7]:
similar_sql=f"""
Q: What is the average monthly charges for customers who have churned?
A: SELECT AVG(monthly_charges) from customer_data where churn=1;

Q:Tell me about customerID 3114822?
A: SELECT * from customer_data where customerID=3114822;

Q:Tell me SHAP contribution of customerID 3114822?
A: SELECT * from customer_shap_contributions where customerID=3114822;

Q:Tell me some action recommendations for customerID 3114822?
A: SELECT * from customer_counterfactuals where customerID=3114822;

Q:What are the top contributors to churn?
A:SELECT `global_shap_summary`.Feature,`global_shap_summary`.`Group`, `global_shap_summary`.`Probability Change (%)`,`global_shap_summary`.`Importance Rank`
 FROM `global_shap_summary` ORDER BY `global_shap_summary`.`Importance Rank` ASC

Q:Give a summary of shap contribution of age and revenue customers in service city hou
A:  SELECT AVG(`customer_shap_contributions`.agehh1),
    AVG(`customer_shap_contributions`.revenue_per_minute)
    FROM `customer_shap_contributions` 
    JOIN `customer_data` ON
    `customer_shap_contributions`.customerid = `customer_data`.customerid 
    WHERE `customer_data`.service_city = 'hou'

"""


tables_schema=sql_agent.get_all_schemas()

In [8]:
with open('results\\tables_schema.txt', 'w') as f:
    f.write(str(tables_schema))

In [117]:
def generate_sql(user_question:str):
    """
    Generates the SQL query based on the user question
    Use this function to create a SQL query to retrive any data user have asked for or to create an answer to user question.

    Parameters
    ----------
        user_question : str
            the user question
    Returns
    -------
        str
            the result sql query generated
    """

    context_prompt = f"""
            You are a Sqlite SQL guru. Write a SQL comformant query for Sqlite that answers the following question while using the provided context to correctly refer to the SQlite tables and the needed column names.

            Guidelines:
            - Join as minimal tables as possible.
            - When joining tables ensure all join columns are the same data_type.
            - Analyze the database and the table schema provided as parameters and undestand the relations (column and table relations).
            - Don't include any comments in code.
            - **Remove ```sql and ``` from the output and generate the SQL in single line.**
            - Tables should be refered to using a fully qualified name with enclosed in ticks (`) e.g. `project_id.owner.table_name`.
            - Use all the non-aggregated columns from the "SELECT" statement while framing "GROUP BY" block.
            - Use ONLY the column names (column_name) mentioned in Table Schema. DO NOT USE any other column names outside of this.
            - Associate column_name mentioned in Table Schema only to the table_name specified under Table Schema.
            - Table names are case sensitive. DO NOT uppercase or lowercase the table names.
            - Always enclose subqueries and union queries in brackets.
            - Refer to the examples provided below, if given.
            - If the question is not clear, answer with : {"Sorry. No information available for this question."}
            - If the question have task to create summary, create the appropritae summary logic
            - Only answer questions relevant to the tables or columns listed in the table schema If a non-related question comes, answer exactly with : {"Sorry. No information available for this question."}
            - When question is about average feature contribution to churn, use customer_shap_contributions columns only in the output unless it's necessary
            - ***When a subset data query is asked DONOT use LIMIT. Return query for whole data asked for***
            - When using a column make sure it's available in the table schema provided.
            - **When using column values to subset make sure the case matches. it is case sensitive.**

            Below are descriptions for the tables in the database:

            customer_data : 
                - Contains all available customer data points and their churn prediction
                - Donot use this data for SHAP contribution except for adding filters
            data_dictionary: 
                - Contains the description of all the columns in the customer_data table
            customer_shap_contributions : 
                - Contains the individual feature contribution towards every customers churn prediction in SHAP scale.
                - High positive SHAP value indicates higher probability of churn and vice versa.
                - Can be used to explain a churn prediction of a customer in combination with customer_data
            customer_counterfactuals : 
                - Contains the counterfactuals generated for every customer. Can be used to provide recommendations to a customer to reduce churn


            Here are some examples of user-question and SQL queries:
            Q: What is the average monthly charges for customers who have churned?
            A: SELECT AVG(monthly_charges) from customer_data where churn=1;

            Q:Tell me about customerID 3114822?
            A: SELECT * from customer_data where customerID=3114822;

            Q:Tell me SHAP contribution of customerID 3114822?
            A: SELECT * from customer_shap_contributions where customerID=3114822;

            Q:Tell me some recommendations to reduce churn for customerID 3114822?
            A: SELECT * from customer_counterfactuals where customerID=3114822;


            Q:Give a summary of shap contribution of age and revenue customers in service city hou
            A:  SELECT AVG(`customer_shap_contributions`.agehh1),
                AVG(`customer_shap_contributions`.revenue_per_minute)
                FROM `customer_shap_contributions` 
                JOIN `customer_data` ON
                `customer_shap_contributions`.customerid = `customer_data`.customerid 
                WHERE `customer_data`.service_city = 'hou'

            Q:Identify customer data in service city hou?
            A: SELECT * FROM `customer_data` WHERE service_city = 'hou'
                
            Q:Give me shap contribution of customers in service city hou
            A: SELECT T1.* FROM `customer_shap_contributions` T1 JOIN `customer_data` T2 ON T1.customerid = T2.customerid WHERE T2.service_city = 'hou'
            
                
            question:
            {user_question}

            Table Schema:
            Tables:

                customer_data
                customer_shap_contributions
                customer_counterfactuals
                data_dictionary
                global_shap_summary


                Schemas:


                customer_data:
                Name: childreninhh, Type: TEXT
                Name: handsetrefurbished, Type: TEXT
                Name: handsetwebcapable, Type: TEXT
                Name: truckowner, Type: TEXT
                Name: rvowner, Type: TEXT
                Name: homeownership, Type: TEXT
                Name: buysviamailorder, Type: TEXT
                Name: respondstomailoffers, Type: TEXT
                Name: optoutmailings, Type: TEXT
                Name: nonustravel, Type: TEXT
                Name: ownscomputer, Type: TEXT
                Name: hascreditcard, Type: TEXT
                Name: newcellphoneuser, Type: TEXT
                Name: notnewcellphoneuser, Type: TEXT
                Name: ownsmotorcycle, Type: TEXT
                Name: madecalltoretentionteam, Type: TEXT
                Name: creditrating, Type: TEXT
                Name: prizmcode, Type: TEXT
                Name: occupation, Type: TEXT
                Name: maritalstatus, Type: TEXT
                Name: service_city, Type: TEXT
                Name: monthlyrevenue, Type: REAL
                Name: monthlyminutes, Type: REAL
                Name: totalrecurringcharge, Type: REAL
                Name: directorassistedcalls, Type: REAL
                Name: overageminutes, Type: REAL
                Name: roamingcalls, Type: REAL
                Name: percchangeminutes, Type: REAL
                Name: percchangerevenues, Type: REAL
                Name: droppedcalls, Type: REAL
                Name: blockedcalls, Type: REAL
                Name: unansweredcalls, Type: REAL
                Name: customercarecalls, Type: REAL
                Name: threewaycalls, Type: REAL
                Name: receivedcalls, Type: REAL
                Name: outboundcalls, Type: REAL
                Name: inboundcalls, Type: REAL
                Name: peakcallsinout, Type: REAL
                Name: offpeakcallsinout, Type: REAL
                Name: droppedblockedcalls, Type: REAL
                Name: callforwardingcalls, Type: REAL
                Name: callwaitingcalls, Type: REAL
                Name: monthsinservice, Type: REAL
                Name: uniquesubs, Type: REAL
                Name: activesubs, Type: REAL
                Name: handsets, Type: REAL
                Name: handsetmodels, Type: REAL
                Name: currentequipmentdays, Type: REAL
                Name: agehh1, Type: REAL
                Name: retentioncalls, Type: REAL
                Name: retentionoffersaccepted, Type: REAL
                Name: referralsmadebysubscriber, Type: REAL
                Name: adjustmentstocreditrating, Type: REAL
                Name: revenue_per_minute, Type: REAL
                Name: total_calls, Type: REAL
                Name: avg_call_duration, Type: REAL
                Name: service_tenure, Type: REAL
                Name: customer_support_interaction, Type: REAL
                Name: handsetprice, Type: REAL
                Name: incomegroup, Type: REAL
                Name: customerid, Type: REAL
                Name: churn, Type: REAL
                Name: prediction, Type: REAL

                    Distinct Values for Categorical Variables:

                    childreninhh: ['no', 'yes']
                    handsetrefurbished: ['no', 'yes']
                    handsetwebcapable: ['no', 'yes']
                    truckowner: ['yes', 'no']
                    rvowner: ['yes', 'no']
                    homeownership: ['known', 'unknown']
                    buysviamailorder: ['no', 'yes']
                    respondstomailoffers: ['yes', 'no']
                    optoutmailings: ['yes', 'no']
                    nonustravel: ['no', 'yes']
                    ownscomputer: ['no', 'yes']
                    hascreditcard: ['no', 'yes']
                    newcellphoneuser: ['no', 'yes']
                    notnewcellphoneuser: ['no', 'yes']
                    ownsmotorcycle: ['no', 'yes']
                    madecalltoretentionteam: ['no', 'yes']
                    creditrating: ['2-high', '3-good', '4-medium', '5-low', '1-highest', '6-verylow', '7-lowest']
                    prizmcode: ['rural', 'other', 'suburban', 'town']
                    occupation: ['professional', 'other', 'homemaker', 'crafts', 'self', 'clerical', 'retired', 'student']
                    maritalstatus: ['yes', 'no', 'unknown']


                customer_shap_contributions:
                Name: childreninhh, Type: REAL
                Name: handsetrefurbished, Type: REAL
                Name: handsetwebcapable, Type: REAL
                Name: truckowner, Type: REAL
                Name: rvowner, Type: REAL
                Name: homeownership, Type: REAL
                Name: buysviamailorder, Type: REAL
                Name: respondstomailoffers, Type: REAL
                Name: optoutmailings, Type: REAL
                Name: nonustravel, Type: REAL
                Name: ownscomputer, Type: REAL
                Name: hascreditcard, Type: REAL
                Name: newcellphoneuser, Type: REAL
                Name: notnewcellphoneuser, Type: REAL
                Name: ownsmotorcycle, Type: REAL
                Name: madecalltoretentionteam, Type: REAL
                Name: creditrating, Type: REAL
                Name: prizmcode, Type: REAL
                Name: occupation, Type: REAL
                Name: maritalstatus, Type: REAL
                Name: service_city, Type: REAL
                Name: monthlyrevenue, Type: REAL
                Name: monthlyminutes, Type: REAL
                Name: totalrecurringcharge, Type: REAL
                Name: directorassistedcalls, Type: REAL
                Name: overageminutes, Type: REAL
                Name: roamingcalls, Type: REAL
                Name: percchangeminutes, Type: REAL
                Name: percchangerevenues, Type: REAL
                Name: droppedcalls, Type: REAL
                Name: blockedcalls, Type: REAL
                Name: unansweredcalls, Type: REAL
                Name: customercarecalls, Type: REAL
                Name: threewaycalls, Type: REAL
                Name: receivedcalls, Type: REAL
                Name: outboundcalls, Type: REAL
                Name: inboundcalls, Type: REAL
                Name: peakcallsinout, Type: REAL
                Name: offpeakcallsinout, Type: REAL
                Name: droppedblockedcalls, Type: REAL
                Name: callforwardingcalls, Type: REAL
                Name: callwaitingcalls, Type: REAL
                Name: monthsinservice, Type: REAL
                Name: uniquesubs, Type: REAL
                Name: activesubs, Type: REAL
                Name: handsets, Type: REAL
                Name: handsetmodels, Type: REAL
                Name: currentequipmentdays, Type: REAL
                Name: agehh1, Type: REAL
                Name: retentioncalls, Type: REAL
                Name: retentionoffersaccepted, Type: REAL
                Name: referralsmadebysubscriber, Type: REAL
                Name: adjustmentstocreditrating, Type: REAL
                Name: revenue_per_minute, Type: REAL
                Name: total_calls, Type: REAL
                Name: avg_call_duration, Type: REAL
                Name: service_tenure, Type: REAL
                Name: customer_support_interaction, Type: REAL
                Name: handsetprice, Type: REAL
                Name: incomegroup, Type: REAL
                Name: customerid, Type: REAL

                    Distinct Values for Categorical Variables:



                customer_counterfactuals:
                Name: customerid, Type: REAL
                Name: changes, Type: TEXT

                    Distinct Values for Categorical Variables:



                data_dictionary:
                Name: Column Name, Type: TEXT
                Name: Data Type, Type: TEXT
                Name: Description, Type: TEXT

                    Distinct Values for Categorical Variables:

                    Data Type: ['object', 'float64', 'int64']



            """
    context_query=gmodel.generate_content(context_prompt, stream=False)
    print(remove_sql_and_backticks(str(context_query.candidates[0].content.parts[0].text)))

    return str(context_query.candidates[0].content.parts[0].text)


def execute_sql(sql_query:str):
    """
    Executes the provided SQL query using the sql_agent and returns the result as a dictionary.

    The function takes a SQL query as input, executes it using the sql_agent, and returns the result as a dictionary.

    Use this only for cases where the answer to user question is directly available.


    Parameters
    ----------
    sql_query : str
        The SQL query to be executed.

    Returns
    -------
    dict
        The result of the SQL query, converted to a dictionary.
    """

    print(sql_query)
    sql_query=remove_sql_and_backticks(sql_query)
    sql_query=sql_query.replace("\\", "")
    print(sql_query)
    rdf=sql_agent.execute_query(query=sql_query)
    #print(rdf.to_dict())
    return f"""retrived answer is : {rdf.to_dict()}
    """
    
def execute_sql_df(sql_query:str):
    """
    Executes the provided SQL query using the sql_agent and returns the result as a dictionary.

    The function takes a SQL query as input, executes it using the sql_agent, and returns the result as a dictionary.

    Use this function when the answer to user question needs to be processed or modified before returning to user.

    Parameters
    ----------
    sql_query : str
        The SQL query to be executed.

    Returns
    -------
    dict
        The result of the SQL query, converted to a dictionary.
    """

    print(sql_query)
    sql_query=remove_sql_and_backticks(sql_query)
    sql_query=sql_query.replace("\\", "")
    print(sql_query)
    rdf=sql_agent.execute_query(query=sql_query)
    return rdf

def global_shap_summary(customer_data_sql_query:str,shap_data_sql_query:str):
    """
        Calculates the global SHAP summary from the customer subset data query and SHAP feature contribution data query for same subset.
        This can be used to identify patterns, top churn contributors and feature importance for the subset of data.
        Requires two inputs:
        ----------
        customer_data_sql_query : str
            The SQL query to retrieve the customer data for a subset of customers as asked by user
        shap_data_sql_query : str
            The SQL query to retrieve the SHAP feature contribution data for the same subset of customers as asked by user
        
        It is important to have both input data to be of same size and same order of customers.

    """
    df_data=execute_sql_df(customer_data_sql_query)
    df_shap_data=execute_sql_df(shap_data_sql_query)


    def sigmoid(x):
        """ Sigmoid function to convert log-odds to probabilities. """
        return 1 / (1 + np.exp(-x))
    
    base_value=1.0    
    base_probability = sigmoid(base_value)
    results = []
    feature_importances = {}
    # Process each feature
    common_columns = df_data.columns.intersection(df_shap_data.columns)
    #Remove if column customerid exists
    common_columns = common_columns.drop('customerid')


    # Calculate feature importances
    for feature in common_columns:
        feature_shap_values = df_shap_data[feature]
        feature_importances[feature] = np.mean(np.abs(feature_shap_values))

    importance_df = pd.DataFrame(list(feature_importances.items()), columns=['Feature', 'Importance'])
    importance_df.sort_values('Importance', ascending=False, inplace=True)
    importance_df['Rank'] = range(1, len(importance_df) + 1)
    importance_ranks = importance_df.set_index('Feature')['Rank'].to_dict()

    for feature in common_columns:
        feature_values = df_data[feature]
        feature_shap_values = df_shap_data[feature]
        df = pd.DataFrame({feature: feature_values, 'SHAP Value': feature_shap_values})
        numeric_features = df_data.select_dtypes(include=['number']).columns

        if feature in numeric_features:
            df['Group'] = pd.qcut(df[feature], 10, duplicates='drop')
        else:
            df['Group'] = df[feature]

        group_avg = df.groupby('Group',observed=True)['SHAP Value'].mean().reset_index()
        group_avg['Adjusted Probability'] = sigmoid(base_value + group_avg['SHAP Value'])
        group_avg['Probability Change (%)'] = (group_avg['Adjusted Probability'] - base_probability) * 100
        group_avg['Feature'] = feature
        group_avg['Feature Importance'] = feature_importances[feature]
        group_avg['Importance Rank'] = importance_ranks[feature]
        results.append(group_avg)
    
    result_df = pd.concat(results, ignore_index=True)
    result_df.sort_values(['Importance Rank', 'Probability Change (%)'], ascending=[True, False], inplace=True)
    result_df=result_df[result_df['Importance Rank']<=10]

    return f"""The SHAP summary from the model is: 
    {tabulate.tabulate(result_df[['Feature','Group','Probability Change (%)','SHAP Value','Importance Rank']], headers='keys', tablefmt='pipe', showindex='never')}
    - 'Group' refers to different sub groups within a feature.
    - 'Probability Change (%)' measures the feature group's probability contribution towards the target compared to base model.
    - 'Feature_Importance' shows the importance of the feature in model predictions; a lower rank means higher importance.
    - SHAP Value : The higher the SHAP value, the more the feature pushes the model output towards loan approval, and vice versa
    - Note : (A,B] means between A and B
    - Use the Group SHAP value and Probability Change (%) to understand the feature importance and its contribution to the model and provide insights to the user.
    - Insights generated for each groups should be grounded with supporting facts from summary 
    - Create a report of with no more than 1000 words to summarize the findings and provide insights to the user.
    """


In [118]:
gen_model = genai.GenerativeModel(
    model_name="gemini-1.5-flash-001",
    tools=[generate_sql,execute_sql,global_shap_summary] # list of all available tools
)
chat = gen_model.start_chat(enable_automatic_function_calling=True)

In [125]:
response = chat.send_message("""Give me top 10 customers with highest churn probability predicted by the model.""")
print(response.candidates[0].content.parts[0].text)

```sql
SELECT T1.customerid, T1.prediction FROM `customer_data` T1 ORDER BY T1.prediction DESC LIMIT 10
```
SELECT T1.customerid, T1.prediction FROM `customer_data` T1 ORDER BY T1.prediction DESC LIMIT 10
SELECT T1.customerid, T1.prediction FROM `customer_data` T1 ORDER BY T1.prediction DESC LIMIT 10
The top 10 customers with the highest churn probability predicted by the model are:

* Customer ID: 3334558, Prediction: 0.90535253
* Customer ID: 3046878, Prediction: 0.87296563
* Customer ID: 3010870, Prediction: 0.8587452
* Customer ID: 3141974, Prediction: 0.85383165
* Customer ID: 3267306, Prediction: 0.8526102
* Customer ID: 3018142, Prediction: 0.8508204
* Customer ID: 3233142, Prediction: 0.8473394
* Customer ID: 3273950, Prediction: 0.83539003
* Customer ID: 3140026, Prediction: 0.8304587
* Customer ID: 3211030, Prediction: 0.8289688 



In [113]:
response = chat.send_message("""Identify top churn contributors in service city hou?""")
print(response.candidates[0].content.parts[0].text)

SELECT T2.childreninhh, T2.handsetrefurbished, T2.handsetwebcapable, T2.truckowner, T2.rvowner, T2.homeownership, T2.buysviamailorder, T2.respondstomailoffers, T2.optoutmailings, T2.nonustravel, T2.ownscomputer, T2.hascreditcard, T2.newcellphoneuser, T2.notnewcellphoneuser, T2.ownsmotorcycle, T2.madecalltoretentionteam, T2.creditrating, T2.prizmcode, T2.occupation, T2.maritalstatus, T2.service_city, T2.monthlyrevenue, T2.monthlyminutes, T2.totalrecurringcharge, T2.directorassistedcalls, T2.overageminutes, T2.roamingcalls, T2.percchangeminutes, T2.percchangerevenues, T2.droppedcalls, T2.blockedcalls, T2.unansweredcalls, T2.customercarecalls, T2.threewaycalls, T2.receivedcalls, T2.outboundcalls, T2.inboundcalls, T2.peakcallsinout, T2.offpeakcallsinout, T2.droppedblockedcalls, T2.callforwardingcalls, T2.callwaitingcalls, T2.monthsinservice, T2.uniquesubs, T2.activesubs, T2.handsets, T2.handsetmodels, T2.currentequipmentdays, T2.agehh1, T2.retentioncalls, T2.retentionoffersaccepted, T2.ref

In [135]:
response = chat.send_message("""What are some recommendations to  reduce churn for customer id 3334558?Give me a list of actions? 
                             Don't repeat same actions.
                             If possible give a approx churn reduction probablity""")
print(response.candidates[0].content.parts[0].text)

SELECT * FROM `customer_counterfactuals` WHERE customerid = 3334558
SELECT * FROM `customer_counterfactuals` WHERE customerid = 3334558
SELECT * FROM `customer_counterfactuals` WHERE customerid = 3334558
Here are some actions to reduce churn for customer ID 3334558:

1. **Increase Active Subscriptions:** This customer has 2 active subscriptions. Increasing it to 4 could decrease churn probability by around 50%. 
2. **Offer Director-Assisted Calls, Call Waiting Calls, and Customer Care Calls:** Offering these services could potentially decrease churn probability by around 50%.
3. **Reduce Revenue Per Minute:** The customer's current revenue per minute is 59.99. Reducing it to 0.2 could also significantly decrease churn probability by around 35%.
4. **Increase Average Call Duration:**  The customer's current average call duration is 0.0. Increasing it to 5.5 could potentially decrease churn probability by around 35%.
5. **Adjust Handset Price and Quantity:**  Reduce the handset price fro

In [140]:
response = chat.send_message("""I am a retention strategist. Give me a report on churn contributors in service city hou with age less than 40 and 5 plans to reduce churn.""")
print(response.candidates[0].content.parts[0].text)

SELECT T2.agehh1, T2.revenue_per_minute FROM `customer_shap_contributions` AS T1 INNER JOIN `customer_data` AS T2 ON T1.customerid = T2.customerid WHERE T2.service_city = 'hou' AND T2.agehh1 < 40

SELECT T2.agehh1, T2.revenue_per_minute FROM `customer_shap_contributions` AS T1 INNER JOIN `customer_data` AS T2 ON T1.customerid = T2.customerid WHERE T2.service_city = \'hou\' AND T2.agehh1 < 40
SELECT T2.agehh1, T2.revenue_per_minute FROM `customer_shap_contributions` AS T1 INNER JOIN `customer_data` AS T2 ON T1.customerid = T2.customerid WHERE T2.service_city = 'hou' AND T2.agehh1 < 40
## Churn Contributors for Customers Under 40 in HOU

This report focuses on churn contributors for customers under 40 years old in the HOU service city,  identifying key factors that influence their churn probability. 

**Key Churn Contributors:**

* **Revenue Per Minute:**  This feature has a significant impact on churn. Customers with higher revenue per minute are more likely to churn.  
* **Current Equi

In [137]:
response = chat.send_message("""Ok. Help me create a campaign based on previous report.""")
print(response.candidates[0].content.parts[0].text)

## Campaign: "Stay Connected, HOUston!" 

**Target Audience:** Customers in service city HOU with:

* **High revenue per minute** 
* **Shorter service tenure** 
* **Lower credit rating**
* **Longer current equipment days**
* **High monthly minutes usage**

**Campaign Objectives:**

* Increase customer satisfaction and loyalty.
* Reduce churn rate within the HOU market.
* Drive revenue by encouraging new subscriptions and upgrades. 

**Campaign Messaging:**

* **Theme:** Stay connected with HOUston's best mobile experience.
* **Headline:** Enjoy unbeatable savings, exclusive perks, and exceptional service.
* **Call to Action:** Upgrade your plan today and experience the difference!

**Campaign Elements:**

**1. Personalized Email Campaign:**

* **Subject:**  Your HOUston mobile plan - Stay connected with special offers!
* **Content:**
    * Highlight exclusive discounts and promotions tailored to their specific needs (e.g., lower revenue per minute, special credit rating offers).
    * 

In [139]:
response = chat.send_message("""I want to do a campaign on Instagram. Based on previous report create a campaign for Instagram addressing the major reasons for churn""")
print(response.candidates[0].content.parts[0].text)

## Instagram Campaign: "HOUston, We Hear You!"

**Target Audience:**  Instagram users in Houston, Texas, who are existing or potential mobile plan customers, especially those who:

* Have recently churned or are at risk of churn.
* Have expressed concerns about pricing, service tenure, or equipment.
*  Show interest in mobile plans, technology, or local events. 

**Campaign Objectives:**

* **Address Key Churn Concerns:**  Acknowledge and directly address the major reasons for churn in HOUston (pricing, service tenure, equipment, etc.).
* **Build Trust and Transparency:**  Demonstrate your commitment to understanding and addressing customer needs.
* **Increase Customer Loyalty:** Retain existing customers by showing them you value their business.
* **Attract New Customers:**  Appeal to potential customers seeking a mobile provider that understands their concerns.

**Campaign Theme:**  **"HOUston, We Hear You!  Your Mobile Experience Matters."**

**Campaign Visuals:**

* **Images & Vide

In [96]:
gen_model._tools.to_proto()

[function_declarations {
   name: "generate_sql"
   description: "\n    Generates the SQL query based on the user question\n    Use this function to create a SQL query to retrive any data user have asked for or to create an answer to user question.\n\n    Parameters\n    ----------\n        user_question : str\n            the user question\n    Returns\n    -------\n        str\n            the result sql query generated\n    "
   parameters {
     type_: OBJECT
     properties {
       key: "user_question"
       value {
         type_: STRING
       }
     }
     required: "user_question"
   }
 }
 function_declarations {
   name: "execute_sql"
   description: "\n    Executes the provided SQL query using the sql_agent and returns the result as a dictionary.\n\n    The function takes a SQL query as input, executes it using the sql_agent, and returns the result as a dictionary.\n\n    Use this only for cases where the answer to user question is directly available.\n\n\n    Parameters\n

In [54]:
sm=global_shap_summary(customer_data_sql_query="SELECT * FROM `customer_data` WHERE service_city = 'hou' "
                       ,shap_data_sql_query="SELECT T1.* FROM `customer_shap_contributions` T1 JOIN `customer_data` T2 ON T1.customerid = T2.customerid WHERE T2.service_city = 'hou'")

sm

SELECT * FROM `customer_data` WHERE service_city = 'hou' 
SELECT * FROM `customer_data` WHERE service_city = 'hou' 
SELECT T1.* FROM `customer_shap_contributions` T1 JOIN `customer_data` T2 ON T1.customerid = T2.customerid WHERE T2.service_city = 'hou'
SELECT T1.* FROM `customer_shap_contributions` T1 JOIN `customer_data` T2 ON T1.customerid = T2.customerid WHERE T2.service_city = 'hou'


,Group,SHAP Value,Adjusted Probability,Probability Change (%),Feature,Feature Importance,Importance Rank
243,"(752.0, 1026.0]",0.324676,0.789959,5.890005,currentequipmentdays,0.263003,1
242,"(599.0, 752.0]",0.223438,0.772668,4.160942,currentequipmentdays,0.263003,1
241,"(461.5, 599.0]",0.178325,0.764647,3.358792,currentequipmentdays,0.263003,1
239,"(341.5, 391.0]",0.167750,0.762738,3.167953,currentequipmentdays,0.263003,1
240,"(391.0, 461.5]",0.117189,0.753467,2.240827,currentequipmentdays,0.263003,1
...,...,...,...,...,...,...,...
10,known,-0.001056,0.730851,-0.020764,homeownership,0.001112,56
23,yes,-0.000187,0.731022,-0.003686,hascreditcard,0.000924,57
22,no,-0.000729,0.730915,-0.014341,hascreditcard,0.000924,57
253,"(-0.001, 1.0]",0.000789,0.731214,0.015505,retentionoffersaccepted,0.000859,58


In [131]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What are some recommendations to  reduce churn for customer id 3269506?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'generate_sql', 'args': {'user_question': 'What are some recommendations to  reduce churn for customer id 3269506?'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'generate_sql', 'response': {'result': 'SELECT * FROM `customer_counterfactuals` WHERE customerid = 3269506; \n'}}}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'execute_sql', 'args': {'sql_query': 'SELECT * FROM `customer_counterfactuals` WHERE customerid = 3269506;'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'execute_sql', 'response': {'result': "retrived answer is : {'customerid': {}, 'changes'